In [8]:
! nvidia-smi

Sun May 22 14:27:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    32W /  70W |   1818MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
!pip install pytorch-tabnet optuna

In [10]:
import pandas as pd
import numpy as np
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score,classification_report
import optuna as opt
import torch
import os
import joblib

In [11]:
def make_save_cv_model(i,model_name,model,best_params,optim,output_path="./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/cross_validated_models"):

    ''' This function saves cross validation model in the corresponding directory ( if the path does not exist it creates the path for it'''


    if os.path.exists(os.path.join(output_path,f"{i}_{model_name}_{optim}")):
        joblib.dump(model, os.path.join(output_path,f"{i}_{model_name}_{optim}/{i}_model.z"))
        with open(os.path.join(output_path,f"{i}_{model_name}_{optim}/model_params.txt"),"w+") as file:
            file.write(str(best_params))
    else:
        os.mkdir(os.path.join(output_path,f"{i}_{model_name}_{optim}"))
        joblib.dump(model, os.path.join(output_path,f"{i}_{model_name}_{optim}/{i}_model.z"))
        with open(os.path.join(output_path,f"{i}_{model_name}_{optim}/model_params.txt"),"w+") as file:
            file.write(str(best_params))

In [12]:
def train(fold_dict,fold,model_name,sc_df,tar_col,optim,optim_trial,k_folds=10,tar_cols="",verbose=1):

    ''' this function is used to train the model with parameters optimization using optuna and cross validation using stratified k_folds'''

    y = sc_df[tar_col]
    x = sc_df.drop([tar_col],axis=1)
    model_name = model_name 
    def objective(trial):
      train_index = fold_dict[fold]["train"]
      test_index = fold_dict[fold]["test"]
      clf = TabNetClassifier(n_d=trial.suggest_int("n_d", 8, 64),
                              n_a =trial.suggest_int("n_a", 8, 64),
                              n_steps = trial.suggest_int("n_steps",3,10),
                              gamma =trial.suggest_float("gamma", 1.0, 2.0),
                              n_independent = trial.suggest_int("n_independent",1,5),
                              n_shared = trial.suggest_int("n_shared",1,5),
                              momentum = trial.suggest_float("momentum", 0.01, 0.4),
                              optimizer_fn = torch.optim.Adam,
                              # scheduler_fn = torch.optim.lr_scheduler,
                              # scheduler_params = {"gamma" :trial.suggest_float("sch-gamma", 0.5, 0.95), "step_size": trial.suggest_int("sch_step_size", 10, 20, 2)},
                              verbose = verbose,
                              device_name = "auto"
                              )
      # print(f" train_index :: {train_index}")
      # print(f" test_index :: {test_index}")
      X_train,X_test = x.iloc[train_index,:], x.iloc[test_index,:]
      # print(X_train.shape, X_test.shape)
      X_train, X_test = X_train.to_numpy(dtype=np.float64), X_test.to_numpy(dtype=np.float64)
      Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
      Y_train, Y_test = Y_train.to_numpy(dtype=np.float64), Y_test.to_numpy(dtype=np.float64)
      print(Y_train.shape, Y_test.shape)
      clf.fit(X_train, Y_train,
              eval_set=[(X_test, Y_test)],
              eval_metric=['accuracy'])
      Y_pred = clf.predict(X_test)
      print(classification_report(Y_test, Y_pred, labels=[x for x in range(6)]))
      acc = accuracy_score(Y_pred, Y_test)
      return acc

    print(f"Starting optimization for fold : [{fold}/{k_folds}]")
    study = opt.create_study(direction='maximize')
    study.optimize(objective, n_trials=optim_trial)
    best_params = study.best_params
    print(f" Best params for fold : [{fold}/{k_folds}]")
    print(best_params)
    joblib.dump(best_params,f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/{model_name}/best_params/comp/fold_{fold}_best_params.z")
    with open(f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/{model_name}/best_params/fold_{fold}_best_params.txt", "w+") as file:file.write(str(best_params))
    print(f"Saved best_params at : outputs/{model_name}/best_params/fold_{fold}_best_params.txt")
    train_index = fold_dict[fold]["train"]
    test_index = fold_dict[fold]["test"]
    X_train,X_test = x.iloc[train_index,:], x.iloc[test_index,:]
    # print(X_train.shape, X_test.shape)
    X_train, X_test = X_train.to_numpy(dtype=np.float64), X_test.to_numpy(dtype=np.float64)
    Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
    Y_train, Y_test = Y_train.to_numpy(dtype=np.float64), Y_test.to_numpy(dtype=np.float64)
    clf_model = TabNetClassifier(**study.best_params)
    clf_model.fit(X_train,Y_train)
    Y_pred = clf_model.predict(X_test)
    clf_report = classification_report(Y_test, Y_pred, labels=[x for x in range(6)])
    with open(f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/classification_report/{model_name}_{fold}_classification_report.txt","w+") as file:file.write(str(clf_report))
    accuracy = accuracy_score(Y_pred, Y_test)
    with open(f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/{model_name}/{model_name}_{fold}_accuracy_score.txt","w+") as file:file.write(f" accuracy :: {str(accuracy)}")
    try:
        print("[++] Saving the model and parameters in corresponding directories")
        make_save_cv_model(fold,model_name,clf_model,best_params,optim=optim)
    except:
        print("[-] Failed to save the model")

In [13]:
use_df = pd.read_csv("./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/data/trainable_scaled_balanced.csv")
tar_col = "PCE_categorical"
model_name = "pytorch_tabnet"
optimizer = "Adam"
fold_dict = joblib.load("./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/inputs/fold_vals/fold_data.z")
fold = 4

In [14]:
train(fold_dict = fold_dict,
      fold = fold,
      model_name=model_name,
      sc_df=use_df,
      tar_col=tar_col,
      optim=optimizer,
      optim_trial = 15)
print(f"[++] Ended the training process for fold {fold}")

[I 2022-05-22 14:27:48,922] A new study created in memory with name: no-name-e29810b2-8f75-4cc0-a6ab-b50e9589c79b


Starting optimization for fold : [4/10]
Device used : cuda
(42138,) (4682,)
epoch 0  | loss: 1.19873 | val_0_accuracy: 0.59569 |  0:00:04s
epoch 1  | loss: 0.87595 | val_0_accuracy: 0.63264 |  0:00:10s
epoch 2  | loss: 0.78657 | val_0_accuracy: 0.65656 |  0:00:12s
epoch 3  | loss: 0.74815 | val_0_accuracy: 0.66254 |  0:00:14s
epoch 4  | loss: 0.72924 | val_0_accuracy: 0.67194 |  0:00:17s
epoch 5  | loss: 0.69596 | val_0_accuracy: 0.6918  |  0:00:19s
epoch 6  | loss: 0.69029 | val_0_accuracy: 0.69799 |  0:00:22s
epoch 7  | loss: 0.68347 | val_0_accuracy: 0.69628 |  0:00:24s
epoch 8  | loss: 0.66416 | val_0_accuracy: 0.7044  |  0:00:26s
epoch 9  | loss: 0.65571 | val_0_accuracy: 0.70333 |  0:00:29s
epoch 10 | loss: 0.64986 | val_0_accuracy: 0.71743 |  0:00:31s
epoch 11 | loss: 0.63999 | val_0_accuracy: 0.72063 |  0:00:33s
epoch 12 | loss: 0.62582 | val_0_accuracy: 0.71123 |  0:00:35s
epoch 13 | loss: 0.62297 | val_0_accuracy: 0.7217  |  0:00:38s
epoch 14 | loss: 0.61866 | val_0_accuracy:

[I 2022-05-22 14:30:50,111] Trial 0 finished with value: 0.7988039299444681 and parameters: {'n_d': 55, 'n_a': 28, 'n_steps': 3, 'gamma': 1.5216955090034765, 'n_independent': 2, 'n_shared': 4, 'momentum': 0.29835297021941737}. Best is trial 0 with value: 0.7988039299444681.


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       826
           1       0.64      0.65      0.64       826
           2       0.83      0.96      0.89       825
           3       1.00      1.00      1.00       825
           4       0.54      0.53      0.54       761
           5       0.72      0.58      0.64       619

    accuracy                           0.80      4682
   macro avg       0.79      0.79      0.79      4682
weighted avg       0.79      0.80      0.79      4682

Saved classification_report at : ./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/classification_report/pytorch_tabnet_4_classification_report.txt
Device used : cuda
(42138,) (4682,)
epoch 0  | loss: 1.41884 | val_0_accuracy: 0.56557 |  0:00:04s
epoch 1  | loss: 1.02837 | val_0_accuracy: 0.59248 |  0:00:09s
epoch 2  | loss: 0.98203 | val_0_accuracy: 0.59163 |  0:00:14s
epoch 3  | loss: 0.92227 | val_0_accuracy: 0.60765 |  0:00:19s
epoch 4  | loss: 0.89137 |

[I 2022-05-22 14:35:08,278] Trial 1 finished with value: 0.7462622810764631 and parameters: {'n_d': 36, 'n_a': 49, 'n_steps': 6, 'gamma': 1.2816732869663265, 'n_independent': 4, 'n_shared': 4, 'momentum': 0.24084035888898794}. Best is trial 0 with value: 0.7988039299444681.


...

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       826
           1       0.52      0.48      0.50       826
           2       0.75      0.87      0.80       825
           3       1.00      1.00      1.00       825
           4       0.45      0.57      0.51       761
           5       0.72      0.39      0.50       619

    accuracy                           0.73      4682
   macro avg       0.74      0.72      0.72      4682
weighted avg       0.74      0.73      0.73      4682

Saved classification_report at : ./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/classification_report/pytorch_tabnet_4_classification_report.txt
Device used : cuda
(42138,) (4682,)
epoch 0  | loss: 1.32246 | val_0_accuracy: 0.61106 |  0:00:02s
epoch 1  | loss: 0.8222  | val_0_accuracy: 0.65143 |  0:00:04s
epoch 2  | loss: 0.75557 | val_0_accuracy: 0.65164 |  0:00:06s
epoch 3  | loss: 0.72516 | val_0_accuracy: 0.6792  |  0:00:09s
epoch 4  | loss: 0.69281 |

[I 2022-05-22 15:27:51,308] Trial 13 finished with value: 0.8131140538231525 and parameters: {'n_d': 43, 'n_a': 57, 'n_steps': 3, 'gamma': 1.6912411164572185, 'n_independent': 1, 'n_shared': 5, 'momentum': 0.01833405710719377}. Best is trial 13 with value: 0.8131140538231525.


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       826
           1       0.66      0.69      0.67       826
           2       0.86      0.95      0.90       825
           3       1.00      1.00      1.00       825
           4       0.59      0.52      0.55       761
           5       0.71      0.66      0.69       619

    accuracy                           0.81      4682
   macro avg       0.80      0.80      0.80      4682
weighted avg       0.81      0.81      0.81      4682

Saved classification_report at : ./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/classification_report/pytorch_tabnet_4_classification_report.txt
Device used : cuda
(42138,) (4682,)
epoch 0  | loss: 1.36747 | val_0_accuracy: 0.60679 |  0:00:02s
epoch 1  | loss: 0.89173 | val_0_accuracy: 0.64161 |  0:00:04s
epoch 2  | loss: 0.77734 | val_0_accuracy: 0.65357 |  0:00:06s
epoch 3  | loss: 0.73538 | val_0_accuracy: 0.66232 |  0:00:09s
epoch 4  | loss: 0.71668 |

[I 2022-05-22 15:30:26,777] Trial 14 finished with value: 0.8015805211448099 and parameters: {'n_d': 44, 'n_a': 42, 'n_steps': 3, 'gamma': 1.7466420917159062, 'n_independent': 1, 'n_shared': 5, 'momentum': 0.02935188825262275}. Best is trial 13 with value: 0.8131140538231525.


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       826
           1       0.66      0.62      0.64       826
           2       0.83      0.96      0.89       825
           3       1.00      1.00      1.00       825
           4       0.53      0.62      0.57       761
           5       0.79      0.53      0.63       619

    accuracy                           0.80      4682
   macro avg       0.80      0.79      0.79      4682
weighted avg       0.81      0.80      0.80      4682

Saved classification_report at : ./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/classification_report/pytorch_tabnet_4_classification_report.txt
 Best params for fold : [4/10]
{'n_d': 43, 'n_a': 57, 'n_steps': 3, 'gamma': 1.6912411164572185, 'n_independent': 1, 'n_shared': 5, 'momentum': 0.01833405710719377}
Saved best_params at : outputs/pytorch_tabnet/best_params/fold_4_best_params.txt
Device used : cuda
No early stopping will be performed, last trainin

Fold 0 has started running on 20-05-22 


Fold 0 has completed sucessfully on 17:00 20-05-22

Fold 1 has started running at 15:15 21-05-22

Fold 2 has started running at 09:45 22-05-22

Fold 2 has completed sucessfully on 10:58 22-05-22

Fold 3 has started running at 18:40 22-05-22

Fold 3 has completed sucessfully on 22-05-22

Fold 4 completed sucessfully on 21:04 on 22-05-22